In [1]:
import pandas as pd
import json

In [2]:
from raw_fetch import download_play_by_play, download_players, download_snap_counts
from pbp_raw_to_play_player import transform_pbp_raw_to_play_player
from play_player_to_game_player import transform_play_player_to_game_player
from t_game_player_to_season_player import transform_game_player_to_season_player
from fantasy import calculate_fantasy_points
from player import merge_player_metadata

In [3]:
fantasy_config = json.load(open('fantasy_config.json'))

In [4]:
player_df = pd.read_parquet('./raw/players/players.parquet')

In [5]:
download_snap_counts(2024)

'c:\\working\\ff_stats_page\\data\\raw\\snap_counts\\snap_counts_2024.parquet'

In [16]:
pbp = pd.read_parquet('./raw/pbp/play_by_play_2024.parquet')

In [17]:
pbp[pbp['fumble'] > 0]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
20,528.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,BUF,home,ARI,...,0.0,0.0,-6.044650,NaN,NaN,NaN,NaN,NaN,0.478197,52.180350
99,2527.0,2024_01_ARI_BUF,2024090801,BUF,ARI,REG,1,ARI,away,BUF,...,0.0,0.0,-4.927136,NaN,NaN,NaN,NaN,NaN,0.691430,30.856955
199,921.0,2024_01_BAL_KC,2024090500,KC,BAL,REG,1,BAL,away,KC,...,0.0,0.0,-5.576134,NaN,NaN,NaN,NaN,NaN,0.457668,54.233199
396,1413.0,2024_01_CAR_NO,2024090806,NO,CAR,REG,1,CAR,away,NO,...,0.0,1.0,-4.797649,NaN,NaN,NaN,NaN,NaN,0.680536,-68.053603
505,4250.0,2024_01_CAR_NO,2024090806,NO,CAR,REG,1,CAR,away,NO,...,0.0,1.0,3.035296,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7743,1239.0,2024_03_PHI_NO,2024092203,NO,PHI,REG,3,PHI,away,NO,...,0.0,1.0,-5.586196,NaN,NaN,NaN,NaN,NaN,0.617259,38.274115
7823,3253.0,2024_03_PHI_NO,2024092203,NO,PHI,REG,3,PHI,away,NO,...,0.0,1.0,2.814311,1.725931,4.812842,3.0,0.634611,0.634611,0.970203,2.979702
7933,1965.0,2024_03_SF_LA,2024092211,LA,SF,REG,3,SF,away,LA,...,0.0,0.0,-3.085736,NaN,NaN,NaN,NaN,NaN,0.967370,3.262955
8027,4434.0,2024_03_SF_LA,2024092211,LA,SF,REG,3,SF,away,LA,...,0.0,0.0,-0.071247,0.393172,3.694635,1.0,1.000000,1.000000,0.973545,2.645463


In [18]:
play_player_raw = transform_pbp_raw_to_play_player(pbp)

In [19]:
play_player_fantasy = calculate_fantasy_points(fantasy_config, play_player_raw)

In [20]:
play_player = merge_player_metadata(
    play_player_fantasy, player_df)

In [21]:
game_player = transform_play_player_to_game_player(play_player)

In [22]:
season_player = transform_game_player_to_season_player(game_player)

## QC Checks


## Save Data


In [13]:
play_player.to_parquet('./tables/play_player_2024.parquet')

In [16]:
game_player.to_parquet('./tables/game_player_2024.parquet')

In [23]:
season_player.to_parquet('./tables/season_player_2024.parquet')

In [25]:
# save to frontend
play_player.to_parquet('../src/data/play_player_2024.parquet', index=False)
game_player.to_parquet('../src/data/game_player_2024.parquet', index=False)
season_player.to_parquet('../src/data/season_player_2024.parquet', index=False)